In [7]:
%pip install tensorflow
%pip install pandas 
%pip install numpy

Note: you may need to restart the kernel to use updated packages.




In [8]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
%autoreload 2

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from feature_engineering.tensor_features import develop_features, floating_conv
from path_finder import path_sorter

from sklearn.preprocessing import StandardScaler
from data_handler import LocalToLargeDataLoader


In [10]:
data_loader = LocalToLargeDataLoader(print_progress=True)
parsed_data = data_loader.load_raw_data(path="../../resources")

Retrieving training data...


In [11]:
print(parsed_data.keys())

Index(['time', 'cog', 'sog', 'rot', 'heading', 'navstat', 'latitude',
       'longitude', 'vesselId', 'portId', 'etaParsed', 'UN_LOCODE', 'ISO',
       'portLongitude', 'portLatitude'],
      dtype='object')


In [12]:
path_dict = path_sorter(parsed_data)

KeyboardInterrupt: 

In [14]:

print(list(path_dict.keys()))
print(path_dict["time_key"][0])
print(path_dict['navstat'])
print(path_dict[list(path_dict.keys())[0]][0])

['rot', 'heading', 'navstat', 'vesselId', 'portId', 'etaParsed', 'time_key', 'group_id', 'time_0', 'cog_0', 'sog_0', 'latitude_0', 'longitude_0', 'time_1', 'cog_1', 'sog_1', 'latitude_1', 'longitude_1', 'time_2', 'cog_2', 'sog_2', 'latitude_2', 'longitude_2', 'time_3', 'cog_3', 'sog_3', 'latitude_3', 'longitude_3', 'time_4', 'cog_4', 'sog_4', 'latitude_4', 'longitude_4', 'time_5', 'cog_5', 'sog_5', 'latitude_5', 'longitude_5', 'time_6', 'cog_6', 'sog_6', 'latitude_6', 'longitude_6', 'time_7', 'cog_7', 'sog_7', 'latitude_7', 'longitude_7', 'time_8', 'cog_8', 'sog_8', 'latitude_8', 'longitude_8', 'time_9', 'cog_9', 'sog_9', 'latitude_9', 'longitude_9']
2024-05-05 13:25:50
0         1
1         0
2         0
3         0
4         0
         ..
152527    0
152528    0
152529    5
152530    0
152531    0
Name: navstat, Length: 152532, dtype: int64
9


In [40]:
features, y, test_features, y_test = data_loader.load_training_data(path_dict) 

Sorting into tests and training data


In [41]:
features=develop_features(features)
test_features=develop_features(test_features)

Developing features with the following columns:  Index(['rot', 'heading', 'navstat', 'etaParsed', 'time_0', 'time_1', 'cog_1',
       'sog_1', 'latitude_1', 'longitude_1', 'time_2', 'cog_2', 'sog_2',
       'latitude_2', 'longitude_2', 'time_3', 'cog_3', 'sog_3', 'latitude_3',
       'longitude_3', 'time_4', 'cog_4', 'sog_4', 'latitude_4', 'longitude_4',
       'time_5', 'cog_5', 'sog_5', 'latitude_5', 'longitude_5', 'time_6',
       'cog_6', 'sog_6', 'latitude_6', 'longitude_6', 'time_7', 'cog_7',
       'sog_7', 'latitude_7', 'longitude_7', 'time_8', 'cog_8', 'sog_8',
       'latitude_8', 'longitude_8', 'time_9', 'cog_9', 'sog_9', 'latitude_9',
       'longitude_9'],
      dtype='object')
Developing features with the following columns:  Index(['rot', 'heading', 'navstat', 'etaParsed', 'time_0', 'time_1', 'cog_1',
       'sog_1', 'latitude_1', 'longitude_1', 'time_2', 'cog_2', 'sog_2',
       'latitude_2', 'longitude_2', 'time_3', 'cog_3', 'sog_3', 'latitude_3',
       'longitude_3', 

In [49]:
print(list(features.keys()))
print(len(list(features.keys())))
# print(features.head())
print(y.head())

['rot', 'heading', 'navstat', 'etaParsed', 'time_0', 'time_1', 'cog_1', 'sog_1', 'latitude_1', 'longitude_1', 'time_2', 'cog_2', 'sog_2', 'latitude_2', 'longitude_2', 'time_3', 'cog_3', 'sog_3', 'latitude_3', 'longitude_3', 'time_4', 'cog_4', 'sog_4', 'latitude_4', 'longitude_4', 'time_5', 'cog_5', 'sog_5', 'latitude_5', 'longitude_5', 'time_6', 'cog_6', 'sog_6', 'latitude_6', 'longitude_6', 'time_7', 'cog_7', 'sog_7', 'latitude_7', 'longitude_7', 'time_8', 'cog_8', 'sog_8', 'latitude_8', 'longitude_8', 'time_9', 'cog_9', 'sog_9', 'latitude_9', 'longitude_9']
50
        latitude_0  longitude_0
39974     49.16460   -123.92530
3942      13.06485    100.88147
14723     32.90688    -79.95385
73623     41.53984    140.65908
100269     1.31259    104.32622


In [51]:

X_train = features.to_numpy()
X_test_np = test_features.to_numpy()
y_train = y.to_numpy()
y_test_np = y_test.to_numpy()


In [52]:
model = Sequential()

# Input layer
input_dim = X_train.shape[1]  # Number of features in your dataset

# Add layers
model.add(Dense(64, input_dim=input_dim, activation='relu'))  # First hidden layer
model.add(Dropout(0.3))  # Dropout for regularization
model.add(Dense(32, activation='relu'))  # Second hidden layer
model.add(Dense(16, activation='relu'))  # Third hidden layer

# Output layer
# If it's a regression task (predicting a continuous variable like time):
model.add(Dense(units=2, activation='linear')) 

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])

c:\Users\Bruker\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [54]:


# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True)


In [55]:
history = model.fit(X_train, y_train, epochs=100, batch_size=32, 
                    validation_split=0.2, callbacks=[early_stopping, model_checkpoint])

Epoch 1/100
3051/3051 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - loss: 1369435668480.0000 - mae: 92748.2188 - val_loss: 3357.0632 - val_mae: 43.4999
Epoch 2/100
3051/3051 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 3362.0422 - mae: 43.5191 - val_loss: 3356.3672 - val_mae: 43.4918
Epoch 3/100
3051/3051 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 3369.2222 - mae: 43.5867 - val_loss: 3353.9158 - val_mae: 43.4631
Epoch 4/100
3051/3051 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 3343.5039 - mae: 43.4030 - val_loss: 3343.0264 - val_mae: 43.3352
Epoch 5/100
3051/3051 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 3321.5706 - mae: 43.1528 - val_loss: 3297.4299 - val_mae: 42.7992
Epoch 6/100
3051/3051 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 3296.8264 - mae: 42.6645 - val_loss: 3168.3083 - val_mae: 41.2322
Epoch 7/100
3051/3051 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 3130.2964 - mae: 40.6615 - val_loss: 2977.8647 - val_mae: 38.6188
Epoch 8/100
3051/3051 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 2922.6570 - mae: 37.

In [56]:
loss, mae = model.evaluate(X_test_np, y_test)
print(f"Test Loss: {loss}, Test MAE: {mae}")

954/954 ━━━━━━━━━━━━━━━━━━━━ 1s 808us/step - loss: 2650.1118 - mae: 31.2926
Test Loss: 2650.480224609375, Test MAE: 31.226856231689453
